EXPLORATION</br>
Bilboard Exploration: </br>
Billboard data is sourced from the billboard web scraping notebook in this repository. See billboard_data_scrape.ipynb before using this notebook.
1. How many songs reached the number 1 position during the sample period </br>
   Will looking at only #1 songs be useful for analysis, or should we look at songs that entered the top 5/10? </br>
2. How many weeks did each of those songs appear on the charts? </br>

Spoitfy Exploration: </br>
Spotify data is sourced from the Spotify Million Playlist Dataset. </br>  
Download the data: https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge </br> 
Save it to the file path: ``spotify-user-playlist-trends\data\Spotify`` </br>

Preparation: </br>
1. Spotify data cleaning: Of 1,000,000 playlists, how many were updated three or fewer times? </br>
2. Artist Name, Track Title of all #1 songs </br>
3. Match Artist Names and Track Titles to Spotify IDs </br>
4. Iterate over Spotify data to identify playlists that have one or more #1 tracks in the playlist </br>
5. Isolate relevant song lines and write to a new Spotify dataframe/file </br>
6. Of x playlists with 3 or fewer edits, how many had a Billboard Charting song on it? </br>

Analysis: </br>
1. Which #1 songs were most present in playlists? Are there any #1 songs that were not on any playlists? </br>
2. Do the top 10 performing billboard songs appear most on the user playlists? </br>
3. For the 10 number one songs that have the most playlist adds, what was the playlist activity in relation to chart activity?</br>

In [ ]:
import json
import pandas as pd
import os
import numpy as np
from datetime import date, timedelta

### Billboard Exploration

In [ ]:
# How many songs reached the number 1 position during the sample period?
# Will looking at only #1 songs be useful for analysis, or should we look at songs that entered the top 5/10? all?
billboard = pd.read_csv('../data/Billboard/billboard_chart_data.csv')

In [ ]:
billboard.info()

In [ ]:
billboard['artist_title'] = billboard.artist+ '-' + billboard.title

In [ ]:
# total number of songs that entered the billboard top 100 during the sample period
billboard.artist_title.nunique()

In [ ]:
# all number 1 position during the sample period
no1 = billboard[billboard.rank_current_week == 1]
no1.to_csv('../data/Billboard/number_1_by_week.csv', index=False)

In [ ]:
#df for #1 song and artist by week
no1_unique = no1[['week_of','artist','title']].drop_duplicates(subset=['artist','title'], keep='first').reset_index()

#trimming artist and song names to better suite key format
no1_unique[['base_artist','expand']] = no1_unique['artist'].str.split(' Featuring ', expand=True)
no1_unique['base_artist'] = no1_unique['base_artist'].str.lower()

#manually updating artist names and titles that do not match across spotify and billboard
no1_unique['base_artist'] = no1_unique['base_artist'].replace({'ke$ha':'kesha', 'far*east movement':'far east movement','luis fonsi & daddy yankee':'luis fonsi'})
no1_unique['title'] = no1_unique['title'].str.lower()
no1_unique['title'] = no1_unique['title'].replace('uptown funk!','uptown funk')

#create a unique identifier that can tie billboard and spotify data
no1_unique['key'] = no1_unique['base_artist'] + '_' + no1_unique['title'].str.lower().str.split(" \(").str[0]

In [ ]:
billboard_key = no1_unique['key'].str.lower().tolist()
billboard_key

In [ ]:
#number of songs that went number one
len(billboard_key)

In [ ]:
artist_list = no1_unique.base_artist.unique().tolist()

In [ ]:
title_list = []
base_list = no1_unique.title.unique().tolist()

for x in base_list:
    x = x.split(' (')
    x = x[0]
    title_list.append(x)

In [ ]:
# number of unique songs that reached #1
len(title_list)

In [ ]:
# all songs that reached at least the number 5 position during the sample period
top5 = billboard[billboard.rank_current_week <= 5]
top5.artist_title.nunique()

In [ ]:
# all songs that reached at least the number 10 position during the sample period
top10 = billboard[billboard.rank_current_week <= 10]
top10.artist_title.nunique()

In [ ]:
#How many weeks did each of those songs appear on the charts?
week_counts = pd.DataFrame(billboard.groupby(['artist','title'])[['artist','title']].value_counts())
week_counts.sort_values('count', ascending=False).head(10)

### Spotify Exploration

In [ ]:
#Of 1,000,000 playlists, how many were updated three or fewer times? 

In [ ]:
#file names - create list of flie strings to leverage in loops
file_list = []                                                    #empty list for strings to land
for file in os.listdir('..\data\Spotify\data'):                   #for loop to locate each file in source folder
    file_name = '..\\data\\Spotify\\data\\' + os.fsdecode(file)   #create a file string name to be read in
    file_list.append(file_name)                                   #add name to list
file_list                                                         #print resulting list

In [ ]:
# get playlist IDs where the playlists were updated three or fewer times
pid_list = []
for file in file_list: #for loop to iterate through files
    with open(file) as data_file:
        d = json.load(data_file)
        playlists = pd.json_normalize(d['playlists'])
        edit_reqs = playlists[playlists.num_edits <= 3]
        pid_list.append(edit_reqs.pid.unique())

In [ ]:
#list of pid arrays to single list of pids
pid_list = np.concatenate(pid_list).ravel().tolist()


In [ ]:
#count of playlists with three or fewer edits
len(pid_list)

In [ ]:
#percent of playlists from Spotify's million dataset
round(len(pid_list) / 1000000 * 100,2)

### Preparation

In [ ]:
# Artist Name, Track Title of all #1 songs
no1_unique[['artist','title']]

#### Spotify Million Playlist Processing need only be completed once. Comment out cells below after program has run and csv has exported.
First cell creates an empty dataframe. </br>

Second cell is a loop that iterates through Spotify Million Playlist Dataset.  The result: </br>
<li> Returns only playlists with three or fewer edits </li>
<li> Returns playlist and track information for instances in which a track that reached No. 1 on the Billboard Charts was added to a user playlist during the sample period </li>
Note the cells above must be executed to generate a list of PIDs (playlist identifiers) and tracks that hit No. 1 (billboard_key)

Third cell checks for number of unique tracks in the results; 79 songs reached number one during the period, so the cell should return 79.

Fourth cell checks the results and prints the title of any track that hit No. 1 on Billboard, but had no occurrences in the Spotify dataframe.

Fifth cell prints results to a .csv


Iterate over Spotify data to identify playlists that have one or more #1 tracks in the playlist </br>
Isolate relevant song lines and write to a new Spotify dataframe/file

In [ ]:
#intiate empty dataframe
top_song_playlists = pd.DataFrame()

In [ ]:
hold = []
for file in file_list: 
    with open(file) as data_file:
        d = json.load(data_file)
        playlists = pd.json_normalize(d['playlists'])
        playlists = playlists[playlists.pid.isin(pid_list)]
        tracks = pd.json_normalize(d, record_path=['playlists','tracks'],meta=[['playlists','pid']])
        tracks = tracks[tracks['playlists.pid'].isin(pid_list)]
        tracks['artist_name'] = tracks['artist_name'].str.lower()
        tracks['artist_name'] = tracks['artist_name'].str.split(',').str[0]
        tracks['artist_name'] = tracks['artist_name'].str.split(" \(").str[0]
        tracks['track_name'] = tracks['track_name'].str.lower()
        tracks['track_name'] = tracks['track_name'].str.split(" \(").str[0]
        tracks['track_name'] = tracks['track_name'].str.split(' -').str[0]
        tracks['key'] = tracks['artist_name'] + '_' + tracks['track_name'].str.lower()
        tracks = tracks[tracks['key'].isin(billboard_key)]
        df = tracks.merge(playlists, how='inner', left_on='playlists.pid', right_on='pid')
        df = df.drop(columns=['tracks','description','playlists.pid'])
        df.modified_at = pd.to_datetime(df.modified_at, unit = 's')
        hold.append(df)
        print(f'file {file} complete')
        
top_song_playlists = pd.concat(hold)

In [ ]:
#check to see if all #1 songs are present in the result dataframe
spotify_tracks = top_song_playlists.track_name.unique().tolist()
len(spotify_tracks)

In [ ]:
tracks.key.unique()

In [ ]:
spotify_tracks.head()

In [ ]:
#check songs present in the billboard top songs list that aren't present in the spotify result set
for x in title_list:
    if x.lower() in spotify_tracks:
        pass
    else:
        print(f'{x} not found')

In [ ]:
#print results
top_song_playlists.to_csv('../data/Spotify/top_song_playlists_tracks.csv')

### #1 Song Analysis

In [ ]:
#read in spotify results csv:
spotify = pd.read_csv('../data/Spotify/top_song_playlists_tracks.csv')
spotify.modified_at = pd.to_datetime(spotify.modified_at)

In [ ]:
#convert all billboard data to compatible format; mimics structure for key creation above
billboard_data = billboard
billboard_data[['base_artist','expand']] = billboard_data['artist'].str.split(' Featuring ', expand=True)
billboard_data['base_artist'] = billboard_data['base_artist'].str.lower()
billboard_data['base_artist'] = billboard_data['base_artist'].replace({'ke$ha':'kesha', 'far*east movement':'far east movement','luis fonsi & daddy yankee':'luis fonsi'})
billboard_data['title'] =billboard_data['title'].replace('Uptown Funk!','uptown funk')
billboard_data['title'] =billboard_data['title'].str.lower()
billboard_data['key'] = billboard_data['base_artist'] + '_' + billboard_data['title'].str.lower().str.split(" \(").str[0]

billboard_data = billboard_data[billboard_data.key.isin(billboard_key)]

In [ ]:
# Of x playlists with 3 or fewer edits, how many had a Billboard Charting song on it?
print('Total playlist count, original dataset: 1000000')
print('Total playlist count, three or fewer edits: ' + str(len(pid_list)))
print('Total playlists with one or more Billboard No. 1 song(s): ' + str(spotify.pid.nunique()))

In [ ]:
#Which artists had the most number one songs on the billboard charts?
billboard_data.groupby(['base_artist'],group_keys=True)[['title']].nunique().sort_values('title', ascending=False) #.head(20)

In [ ]:
#Which artists had the most #1 songs in playlists?
spotify.groupby(['artist_name'],group_keys=True)[['track_name']].nunique().sort_values('track_name', ascending=False)

In [ ]:
#Which songs stayed #1 longest?
weeks_on_chart_rank = billboard_data.groupby(['base_artist', 'title', 'key'],group_keys=True)[['week_of']].nunique().sort_values('week_of', ascending=False).reset_index()
weeks_on_chart_rank

In [ ]:
#Which #1 songs were most present in playlists? 
s_track_count = spotify.groupby(['artist_name', 'track_name'],group_keys=True)[['pid']].count().sort_values('pid', ascending=False).reset_index()
s_track_count

#Are there any #1 songs that were not on any playlists?
#No - our count of 90 unique titles shows this.

In [ ]:
#10 Top Performing Billboard songs
b_top10 = weeks_on_chart_rank[['base_artist','title','key']].head(10)
b_top10

In [ ]:
#Do the top 10 performing billboard songs appear most on the user playlists?
#10 Top Performing (most playlisted) songs
s_top10 = spotify.groupby(['artist_name', 'track_name','key'],group_keys=True)[['pid']].count().sort_values('pid', ascending=False).reset_index().head(10)
s_top10

In [ ]:
b_toplist = b_top10.title.str.lower().tolist()
s_toplist = s_top10.track_name.str.lower().tolist()

b_keys = b_top10.key.str.lower().tolist()
s_keys = s_top10.key.str.lower().tolist()

In [ ]:
#which songs were top performers on billboard and spotify?
def both(b_list, s_list):
    return [x for x in b_list if x in s_list]

both(b_toplist, s_toplist)

In [ ]:
#what are the titles of the top performing songs across billboard and spotify?
top_song_list = list(set(b_toplist + s_toplist))
top_song_keys = list(set(b_keys + s_keys))

In [ ]:
len(top_song_list)

### Visualization Exports

In [ ]:
#billboard data export - top 10 on billboard by weeks at number 1
billboard_data[billboard_data.key.isin(b_keys)].to_csv('../viz exports/billboard_top_performers.csv', index=False)

In [ ]:
#spotify data export - top 10 on spotify by playlist count
spotify[spotify.key.isin(s_keys)].to_csv('../viz exports/spotify_top_perfomrers.csv', index=False)

In [ ]:
#billboard data export - all 18 top performers across spotify and billboard
billboard18 =  billboard_data[billboard_data.key.isin(top_song_keys)]
billboard18.to_csv('../viz exports/all_top_performers_billboard.csv', index=False)

In [ ]:
#spotify data export - all 18 top performers across spotify and billboard
spotify18 = spotify[spotify.key.isin(top_song_keys)] 

In [ ]:
# Billboard data is grouped by week and spotify playlist updates are on a given day.  Need to aggregate spotify updates by week.
# Append the spotify dataframe with a "week of" grouping to sum playlist modifications over the billboard reporting week.

#function - creates a date string for every 7 days from start date, sourced from billboard notebook
def daterange(start_date, end_date):
    for n in range(0, int((end_date - start_date).days) +1, 7):
        yield start_date + timedelta(n)
        
#initiation - list
billboard_weeks=[]

# intitiation - parameters
start = date(2010,1,9)
end = date(2017,12,30)

# append loop
for dt in daterange(start, end):
     billboard_weeks.append(str(dt.strftime('%Y-%m-%d')))

In [ ]:
billboard_weeks

In [ ]:
billboard_week_df = pd.DataFrame(billboard_weeks, columns=["week_of"])
billboard_week_df["week_of"] = pd.to_datetime(billboard_week_df["week_of"])
billboard_week_df["Int Week"] = billboard_week_df["week_of"].apply(lambda x: x.weekofyear)
billboard_week_df["Int Year"] = billboard_week_df["week_of"].apply(lambda x: x.year)

spotify18["modified_at"] = pd.to_datetime(spotify18["modified_at"])
spotify18["Int Week"] = spotify18["modified_at"].apply(lambda x: x.weekofyear)
spotify18["Int Year"] = spotify18["modified_at"].apply(lambda x: x.year)
spotify18[["modified_at", "Int Week", "Int Year"]]

spotify18 = spotify18.merge(billboard_week_df, on=["Int Week", "Int Year"], how="left")

In [ ]:
spotify18.to_csv('../viz exports/all_top_performers_spotify.csv')

In [ ]:
#billboard data read in - all top perfomers
topspotify = pd.read_csv('../viz exports/all_top_performers_spotify.csv')
topspotify.modified_at = pd.to_datetime(topspotify.modified_at)

In [ ]:
#billboard data read in - all top perfomers
topbillboard = pd.read_csv('../viz exports/all_top_performers_billboard.csv')
topbillboard.week_of = pd.to_datetime(topbillboard.week_of)

In [ ]:
spotify_update_count = topspotify[["key", "week_of","modified_at"]]

In [ ]:
spotify_update_count = pd.DataFrame(spotify_update_count.groupby(["key", "week_of"])[["modified_at"]].count().reset_index())

In [ ]:
spotify_update_count.to_csv('../viz exports/spotify_updates_by_week.csv', index=False)

In [ ]:
#ad hoc analysis q: playlists with most followers
topspotify[['name', 'num_followers']].sort_values('num_followers', ascending=False)